In [64]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [65]:
import numpy as np

from scipy import optimize
from deap.benchmarks import schwefel

from abc import ABCMeta
from abc import abstractmethod
from six import add_metaclass


@add_metaclass(ABCMeta)
class ObjectiveFunction(object):

    def __init__(self, name, dim, minf, maxf):
        self.name = name
        self.dim = dim
        self.minf = minf
        self.maxf = maxf

    def sample(self):
        return np.random.uniform(low=self.minf, high=self.maxf, size=self.dim)

    def custom_sample(self):
        return np.repeat(self.minf, repeats=self.dim) \
               + np.random.uniform(low=0, high=1, size=self.dim) *\
               np.repeat(self.maxf - self.minf, repeats=self.dim)

    @abstractmethod
    def evaluate(self, x):
        pass


class Sphere(ObjectiveFunction):

    def __init__(self, dim):
        super(Sphere, self).__init__('Sphere', dim, -100.0, 100.0)

    def evaluate(self, x):
        return sum(np.power(x, 2))


class Rosenbrock(ObjectiveFunction):

    def __init__(self, dim):
        super(Rosenbrock, self).__init__('Rosenbrock', dim, -30.0, 30.0)

    def evaluate(self, x):
        return optimize.rosen(x)


class Rastrigin(ObjectiveFunction):

    def __init__(self, dim):
        super(Rastrigin, self).__init__('Rastrigin', dim, -5.12, 5.12)

    def evaluate(self, x):
        return 10 * len(x)\
               + np.sum(np.power(x, 2) - 10 * np.cos(2 * np.pi * np.array(x)))


class Schwefel(ObjectiveFunction):

    def __init__(self, dim):
        super(Schwefel, self).__init__('Schwefel', dim, -500.0, 500.0)

    def evaluate(self, x):
        return schwefel(x)[0]

In [66]:
#ARTIFICAL BEE
import numpy as np
from abc import ABCMeta
from six import add_metaclass
from copy import deepcopy

@add_metaclass(ABCMeta)
class ArtificialBee:
    TRIAL_DEFAULT_VALUE=0
    DEFAULT_PROBABILITY=0.0
    
    def __init__(self,objective_function):
        #intialising the position of food source where bee is present
        self.position=objective_function.custom_sample() 
        #specifying the objective funtion
        self.objective_function=objective_function
        #setting the boundaries of each food source attributes(variables)
        self.minf=objective_function.minf
        self.maxf=objective_function.maxf
        #setting the fitness value
        self.fitness=objective_function.evaluate(self.position)
        #intialising trial value
        self.trial=ArtificialBee.TRIAL_DEFAULT_VALUE
        #intialising probability 
        self.probability=ArtificialBee.DEFAULT_PROBABILITY
     
    #checking for boundary violation
    def evaluate_boundaries(self,position):
        if(position<self.minf).any() or (pos>self.maxf).any():
            position[position>self.maxf]=self.maxf
            position[position<self.minf]=self.minf
        return pos
    
    #function to update food source based on fitness value
    def update_bee(self,position,fitness):
        if fitness<=self.fitness:
            self.position=position
            self.fitness=fitness
            self.trial=0
        else:
            self.trial+=1;
    
#function to decide if food source should be reset
    def reset_bee(self,max_trials):
        if self.trial>=max_trials:
            self._reset_bee()

#function to reset the food source when the trial counter exceeds limit
    def __reset_bee(self):
        self.position=self.objective_function.custom_sample()
        self.fitness=self.objective_function.evaluate(self.position)
        self.trial = ArtificialBee.TRIAL_INITIAL_DEFAULT_VALUE
        self.probability = ArtificialBee.INITIAL_DEFAULT_PROBABILITY

In [67]:
#EMPLOYEE BEE PHASE
class EmployeeBee(ArtificialBee):
    
    def explore(self,max_trials):
        if self.trial<=max_trial:
            #selecting a partner food source/solution randomly
            partner=np.random.choice(self.position)
            #generating a random value for phi between -1 and 1
            phi=np.random.uniform(low=-1,high=1,size=len(self.position))
            #generating new solution
            new_position=self.position+(self.position-component)*phi
            #checking for boundary violations
            new_position=self.evaluate_boundaries(new_position)
            #fitness value of new solution
            new_fitness=self.objective_function.evaluate(new_position)
            #updating the food source fitness value qualifies
            self.update_bee(new_position,new_fitness)
    
    #computation of fitness value
    def get_fitness(self):
        if self.fitness>=0:
            return 1/(1+self.fitness)
        else:
            return 1+np.abs(self.fitness)
    
    #probability computation
    def compute_probability(self,max_fitness):
        self.probability=self.get_fitness()/max_fitness

In [68]:
#ONLOOOKER BEE PHASE
class OnLookerBee(ArtificialBee):
    
    def onloook(self,best_food_sources,max_trials):
        candidate=np.random,choice(best_food_sources)
        self.__exploit(candidate.position,candidate.fitnesss,max_trials)
    
    def __exploit(self,candidate,fitness,max_trials):
        if self.trial<=max_trial:
            #selecting a partner food source/solution randomly
            partner=np.random.choice(candidate)
            #generating a random value for phi between -1 and 1
            phi=np.random.uniform(low=-1,high=1,size=len(self.position))
            #generating new solution
            new_position=candidate+(candidate-component)*phi
            #checking for boundary violations
            new_position=self.evaluate_boundaries(new_position)
            #fitness value of new solution
            new_fitness=self.objective_function.evaluate(new_position)
        
        if new_fitness<=fitness:
            self.position=new_position
            self.fitness=new_fitness
            self.trial=0
        else:
            self.trial+=1

In [69]:
#Artificial Bee Colony Algorithm 
class ABC:
    def __init__(self,objective_function,colony_size=30,no_of_iteration=5000,max_trials=100):
        self.colony_size=colony_size
        self.objective_function=objective_function
        self.no_of_iteration = no_of_iteration
        self.max_trials = max_trials
        self.optimal_solution = None
        self.optimality_tracking = []
        
    def __reset_algorithm(self):
        self.optimal_solution=None
        self.optimality_tracking = []
        
    def __update_optimality_tracking(self):
        self.optimality_tracking.append(self.optimal_solution.fitness)
        
    def __update_optimal_solution(self):
        new_optimal_solution=min(self.onlooker_bees+self.employee_bees,key=lambda bee: bee.fitness)
        
        if not self.optimal_solution:
            self.optimal_solution=deepcopy(new_optimal_solution)
        else:
            if new_optimal_solution.fitness<self.optimal_solution.fitness:
                self.optimal_solution=deepcopy(new_optimal_solution)
    
    def __initialize_employees(self):
        self.employee_bees=[]
        for i in range(self.colony_size//2):
            self.employee_bees.append(EmployeeBee(self.objective_function))
    
    def __initialize_onlookers(self):
        self.onlooker_bees=[]
        for i in range(self.colony_size//2):
            self.onlooker_bees.append(OnLookerBee(self.objective_function))
            
    def __calculate_probabilities(self):
        sum_fitness = sum(map(lambda bee: bee.get_fitness(), self.employee_bees))
        map(lambda bee: bee.compute_probability(sum_fitness), self.employee_bees)

    def __select_best_food_sources(self):
        self.best_food_sources =\
            filter(lambda bee: bee.prob > np.random.uniform(low=0, high=1),
                   self.employee_bees)
        while not self.best_food_sources:
            self.best_food_sources = \
                filter(lambda bee: bee.prob > np.random.uniform(low=0, high=1),
                       self.employee_bees)
    
    def __employee_bees_phase(self):
        map(lambda bee: bee.explore(self.max_trials), self.employee_bees)
        
    def __onlooker_bees_phase(self):
        map(lambda bee: bee.onlook(self.best_food_sources, self.max_trials),self.onlooker_bees)

    def __scout_bees_phase(self):
        map(lambda bee: bee.reset_bee(self.max_trials),self.onlooker_bees + self.employee_bees)

    def optimize(self):
        self.__reset_algorithm()
        self.__initialize_employees()
        self.__initialize_onlookers()
        for itr in range(self.no_of_iteration):
            self.__employee_bees_phase()
            self.__update_optimal_solution()

            self.__calculate_probabilities()
            self.__select_best_food_sources()

            self.__onlooker_bees_phase()
            self.__scout_bees_phase()

            self.__update_optimal_solution()
            self.__update_optimality_tracking()
            print("iter: {} = cost: {}"
                  .format(itr, "%04.03e" % self.optimal_solution.fitness))

In [70]:
import numpy as np
import matplotlib.pyplot as plt

#from abc import ABC

from matplotlib.style import use

#from objection_function import Rastrigin
#from objection_function import Rosenbrock
#from objection_function import Sphere
#from objection_function import Schwefel


#use('classic')


def get_objective(objective, dimension=30):
    objectives = {'Sphere': Sphere(dimension),
                  'Rastrigin': Rastrigin(dimension),
                  'Rosenbrock': Rosenbrock(dimension),
                  'Schwefel': Schwefel(dimension)}
    return objectives[objective]


def simulate(objective_function, colony_size=30, no_of_iteration=5000,
             max_trials=100, simulations=30):
    itr = range(no_of_iteration)
    values = np.zeros(no_of_iteration)
    box_optimal = []
    for _ in range(simulations):
        optimizer = ABC(objective_function=get_objective(objective_function),
                        colony_size=colony_size, no_of_iteration=no_of_iteration,
                        max_trials=max_trials)
        optimizer.optimize()
        values += np.array(optimizer.optimality_tracking)
        box_optimal.append(optimizer.optimal_solution.fitness)
        print(optimizer.optimal_solution.position)
    values /= simulations

    plt.plot(itr, values, lw=0.5, label=objective_function)
    plt.legend(loc='upper right')


def main():
    plt.figure(figsize=(10, 7))
    simulate('Rastrigin')
    plt.ticklabel_format(axis='y', style='sci', scilimits=(-2, 2))
    plt.xticks(rotation=45)
    plt.show()


if __name__ == '__main__':
    main()

iter: 0 = cost: 4.463e+02
iter: 1 = cost: 4.463e+02
iter: 2 = cost: 4.463e+02
iter: 3 = cost: 4.463e+02
iter: 4 = cost: 4.463e+02
iter: 5 = cost: 4.463e+02
iter: 6 = cost: 4.463e+02
iter: 7 = cost: 4.463e+02
iter: 8 = cost: 4.463e+02
iter: 9 = cost: 4.463e+02
iter: 10 = cost: 4.463e+02
iter: 11 = cost: 4.463e+02
iter: 12 = cost: 4.463e+02
iter: 13 = cost: 4.463e+02
iter: 14 = cost: 4.463e+02
iter: 15 = cost: 4.463e+02
iter: 16 = cost: 4.463e+02
iter: 17 = cost: 4.463e+02
iter: 18 = cost: 4.463e+02
iter: 19 = cost: 4.463e+02
iter: 20 = cost: 4.463e+02
iter: 21 = cost: 4.463e+02
iter: 22 = cost: 4.463e+02
iter: 23 = cost: 4.463e+02
iter: 24 = cost: 4.463e+02
iter: 25 = cost: 4.463e+02
iter: 26 = cost: 4.463e+02
iter: 27 = cost: 4.463e+02
iter: 28 = cost: 4.463e+02
iter: 29 = cost: 4.463e+02
iter: 30 = cost: 4.463e+02
iter: 31 = cost: 4.463e+02
iter: 32 = cost: 4.463e+02
iter: 33 = cost: 4.463e+02
iter: 34 = cost: 4.463e+02
iter: 35 = cost: 4.463e+02
iter: 36 = cost: 4.463e+02
iter: 37 = 

iter: 1352 = cost: 4.463e+02
iter: 1353 = cost: 4.463e+02
iter: 1354 = cost: 4.463e+02
iter: 1355 = cost: 4.463e+02
iter: 1356 = cost: 4.463e+02
iter: 1357 = cost: 4.463e+02
iter: 1358 = cost: 4.463e+02
iter: 1359 = cost: 4.463e+02
iter: 1360 = cost: 4.463e+02
iter: 1361 = cost: 4.463e+02
iter: 1362 = cost: 4.463e+02
iter: 1363 = cost: 4.463e+02
iter: 1364 = cost: 4.463e+02
iter: 1365 = cost: 4.463e+02
iter: 1366 = cost: 4.463e+02
iter: 1367 = cost: 4.463e+02
iter: 1368 = cost: 4.463e+02
iter: 1369 = cost: 4.463e+02
iter: 1370 = cost: 4.463e+02
iter: 1371 = cost: 4.463e+02
iter: 1372 = cost: 4.463e+02
iter: 1373 = cost: 4.463e+02
iter: 1374 = cost: 4.463e+02
iter: 1375 = cost: 4.463e+02
iter: 1376 = cost: 4.463e+02
iter: 1377 = cost: 4.463e+02
iter: 1378 = cost: 4.463e+02
iter: 1379 = cost: 4.463e+02
iter: 1380 = cost: 4.463e+02
iter: 1381 = cost: 4.463e+02
iter: 1382 = cost: 4.463e+02
iter: 1383 = cost: 4.463e+02
iter: 1384 = cost: 4.463e+02
iter: 1385 = cost: 4.463e+02
iter: 1386 = c

iter: 2309 = cost: 4.463e+02
iter: 2310 = cost: 4.463e+02
iter: 2311 = cost: 4.463e+02
iter: 2312 = cost: 4.463e+02
iter: 2313 = cost: 4.463e+02
iter: 2314 = cost: 4.463e+02
iter: 2315 = cost: 4.463e+02
iter: 2316 = cost: 4.463e+02
iter: 2317 = cost: 4.463e+02
iter: 2318 = cost: 4.463e+02
iter: 2319 = cost: 4.463e+02
iter: 2320 = cost: 4.463e+02
iter: 2321 = cost: 4.463e+02
iter: 2322 = cost: 4.463e+02
iter: 2323 = cost: 4.463e+02
iter: 2324 = cost: 4.463e+02
iter: 2325 = cost: 4.463e+02
iter: 2326 = cost: 4.463e+02
iter: 2327 = cost: 4.463e+02
iter: 2328 = cost: 4.463e+02
iter: 2329 = cost: 4.463e+02
iter: 2330 = cost: 4.463e+02
iter: 2331 = cost: 4.463e+02
iter: 2332 = cost: 4.463e+02
iter: 2333 = cost: 4.463e+02
iter: 2334 = cost: 4.463e+02
iter: 2335 = cost: 4.463e+02
iter: 2336 = cost: 4.463e+02
iter: 2337 = cost: 4.463e+02
iter: 2338 = cost: 4.463e+02
iter: 2339 = cost: 4.463e+02
iter: 2340 = cost: 4.463e+02
iter: 2341 = cost: 4.463e+02
iter: 2342 = cost: 4.463e+02
iter: 2343 = c

iter: 3555 = cost: 4.463e+02
iter: 3556 = cost: 4.463e+02
iter: 3557 = cost: 4.463e+02
iter: 3558 = cost: 4.463e+02
iter: 3559 = cost: 4.463e+02
iter: 3560 = cost: 4.463e+02
iter: 3561 = cost: 4.463e+02
iter: 3562 = cost: 4.463e+02
iter: 3563 = cost: 4.463e+02
iter: 3564 = cost: 4.463e+02
iter: 3565 = cost: 4.463e+02
iter: 3566 = cost: 4.463e+02
iter: 3567 = cost: 4.463e+02
iter: 3568 = cost: 4.463e+02
iter: 3569 = cost: 4.463e+02
iter: 3570 = cost: 4.463e+02
iter: 3571 = cost: 4.463e+02
iter: 3572 = cost: 4.463e+02
iter: 3573 = cost: 4.463e+02
iter: 3574 = cost: 4.463e+02
iter: 3575 = cost: 4.463e+02
iter: 3576 = cost: 4.463e+02
iter: 3577 = cost: 4.463e+02
iter: 3578 = cost: 4.463e+02
iter: 3579 = cost: 4.463e+02
iter: 3580 = cost: 4.463e+02
iter: 3581 = cost: 4.463e+02
iter: 3582 = cost: 4.463e+02
iter: 3583 = cost: 4.463e+02
iter: 3584 = cost: 4.463e+02
iter: 3585 = cost: 4.463e+02
iter: 3586 = cost: 4.463e+02
iter: 3587 = cost: 4.463e+02
iter: 3588 = cost: 4.463e+02
iter: 3589 = c

iter: 4559 = cost: 4.463e+02
iter: 4560 = cost: 4.463e+02
iter: 4561 = cost: 4.463e+02
iter: 4562 = cost: 4.463e+02
iter: 4563 = cost: 4.463e+02
iter: 4564 = cost: 4.463e+02
iter: 4565 = cost: 4.463e+02
iter: 4566 = cost: 4.463e+02
iter: 4567 = cost: 4.463e+02
iter: 4568 = cost: 4.463e+02
iter: 4569 = cost: 4.463e+02
iter: 4570 = cost: 4.463e+02
iter: 4571 = cost: 4.463e+02
iter: 4572 = cost: 4.463e+02
iter: 4573 = cost: 4.463e+02
iter: 4574 = cost: 4.463e+02
iter: 4575 = cost: 4.463e+02
iter: 4576 = cost: 4.463e+02
iter: 4577 = cost: 4.463e+02
iter: 4578 = cost: 4.463e+02
iter: 4579 = cost: 4.463e+02
iter: 4580 = cost: 4.463e+02
iter: 4581 = cost: 4.463e+02
iter: 4582 = cost: 4.463e+02
iter: 4583 = cost: 4.463e+02
iter: 4584 = cost: 4.463e+02
iter: 4585 = cost: 4.463e+02
iter: 4586 = cost: 4.463e+02
iter: 4587 = cost: 4.463e+02
iter: 4588 = cost: 4.463e+02
iter: 4589 = cost: 4.463e+02
iter: 4590 = cost: 4.463e+02
iter: 4591 = cost: 4.463e+02
iter: 4592 = cost: 4.463e+02
iter: 4593 = c

AttributeError: 'OnLookerBee' object has no attribute 'pos'

<Figure size 720x504 with 0 Axes>